## 1. introduction & setup

we finetune phi 3 mini 128k instruct on a hindi question answering dataset[1]

[1] Indic QA Hindi dataset, https://huggingface.co/datasets/sepiatone/ai4bharat-IndicQA-hi-202410

#### 1a. we first install some required libraries

In [ ]:
!pip install -qU datasets
!pip install -qU bitsandbytes
!pip install -qU peft
!pip install -qU trl

#### 1b. we log into huggingface

In [ ]:
from google.colab import userdata
import os

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

## 2. prepare the dataset

#### 2a. load the dataset

In [ ]:
from datasets import load_dataset_builder, load_dataset

ds = load_dataset("sepiatone/ai4bharat-IndicQA-hi-202410", split = "train")


# test
print(ds)

Dataset({
    features: ['version', 'data'],
    num_rows: 261
})


#### 2b. prepare the dataset

In [ ]:
def process_data_sample(example):
    processed_example = ()
    context = example["data"]["paragraphs"][0]["context"]

    for j in range(len(example["data"]["paragraphs"][0]["qas"])):
      question = example["data"]["paragraphs"][0]["qas"][j]["question"]
      answer = example["data"]["paragraphs"][0]["qas"][j]["answers"][0]["text"]


      # Prepare the processed example for a History Question Answering System
      processed_example = processed_example + (
          f"{context}\n\n"
          f"उपयोगकर्ता प्रश्न:\n{question}\n\n"
          f"उत्तर:\n{answer}\n\n",
      )

    processed_example = "".join(processed_example)

    return processed_example

In [ ]:
ds = ds.map(lambda example: {'text': process_data_sample(example)}, remove_columns=['version', 'data'])


# test
for i in range(2):
  print(ds[i]["text"])

"दलाई लामा ने अपनी संवेदना और प्रार्थना व्यक्त की और कलाम की मौत को "एक अपूरणीय क्षति" बुला, अपना दुख व्यक्त किया। उन्होंने यह भी कहा, "अनेक वर्षों में, मुझे कई अवसरों पर कलाम के साथ बातचीत करने का मौका मिला। वह एक महान वैज्ञानिक, शिक्षाविद और राजनेता ही नहीं, बल्कि वे एक वास्तविक सज्जन थे, और हमेशा मैंने उनकी सादगी और विनम्रता की प्रशंसा की है। मैंने सामान्य हितों के विषयों की एक विस्तृत श्रृंखला पर हमारी चर्चाओं का आनंद लिया, लेकिन विज्ञान, अध्यात्म और शिक्षा के साथ मुख्य रूप से हमारे बीच चिंतन किया जाता था। " दक्षिण एशियाई नेताओं ने अपनी संवेदना व्यक्त की और दिवंगत राजनेता की सराहना की। भूटान सरकार ने कलाम की मौत के शोक के लिए देश के झंडे को आधी ऊंचाई पर फहराने के लिए आदेश दिया, और श्रद्धांजलि में 1000 मक्खन के दीपक की भेंट किए। भूटान के प्रधानमंत्री शेरिंग तोबगे ने कलाम के प्रति अपना गहरा दुख व्यक्त करते हुए कहा, "वे एक महान नेता थे जिनकी सभी ने प्रशंसा की विशेषकर भारत के युवाओं के वे प्रशंसनीय नेता थे जिन्हें वे जनता का राष्ट्रपति बुलाते थे। "बांग्लादेश की प्रधानमंत्री शेख हसीना न

## 3. model configuration and loading

we load the model and set the configuration parameters

In [ ]:
import torch
import bitsandbytes
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

model_name = 'microsoft/Phi-3-mini-128k-instruct'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

#### 3b. setup the tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

## 4. peft and lora configuration

In [ ]:
from peft import LoraConfig #, prepare_model_for_kbit_training, get_peft_model,PeftModel

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    bias='none',
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

output_dir = "./phi-3-mini-sft-indicqa-hindi"

# Training Arguments
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=True,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True
)

# SFTTrainer Arguments
trainer = SFTTrainer(
    model=model,
    train_dataset=ds, #train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field='text',
    args=training_arguments,
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=None
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/261 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


#### 4b. let's finetune on the dataset!

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
25,1.331400
50,1.257300


TrainOutput(global_step=66, training_loss=1.2805713306773792, metrics={'train_runtime': 646.439, 'train_samples_per_second': 0.404, 'train_steps_per_second': 0.102, 'total_flos': 5983760610754560.0, 'train_loss': 1.2805713306773792, 'epoch': 1.0})

## 5. post training

#### 5a. push to hugging face

In [ ]:
trainer.push_to_hub(commit_message = "end of finetuning", model_name = "phi-3-mini-sft-indicqa-hindi-v0.1")

events.out.tfevents.1730021233.6753c6390e38.20796.2:   0%|          | 0.00/9.28k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/35.7M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/sepiatone/phi-3-mini-sft-indicqa-hindi/commit/0d30a4d476e1c0394a7a2851e2385b02949f9b18', commit_message='end of finetuning', commit_description='', oid='0d30a4d476e1c0394a7a2851e2385b02949f9b18', pr_url=None, repo_url=RepoUrl('https://huggingface.co/sepiatone/phi-3-mini-sft-indicqa-hindi', endpoint='https://huggingface.co', repo_type='model', repo_id='sepiatone/phi-3-mini-sft-indicqa-hindi'), pr_revision=None, pr_num=None)